<a href="https://colab.research.google.com/github/Fuenfgeld/Agent_Tutorial_PydanticAI/blob/main/02_StructuredData_ResultValidators_PydanticAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip -q install pydantic-ai
%pip -q install nest_asyncio
%pip -q install logfire

In [ ]:
import os
from google.colab import userdata

keyAntropic = userdata.get('Claude')
keyOpenAI = userdata.get('openAI')
keyLogFire = userdata.get('logfire')


os.environ["OPENAI_API_KEY"] = keyOpenAI
os.environ["ANTHROPIC_API_KEY"] = keyAntropic

import nest_asyncio
nest_asyncio.apply()
#logfire.configure(token=keyLogFire)

# Structured Data models as output
https://ai.pydantic.dev/output/#structured-output

In [ ]:
import os
from pydantic_ai import Agent
from pydantic_ai.models.openai import OpenAIModel
from pydantic import BaseModel

# Define the model
model = OpenAIModel('gpt-4o-mini')

# Define the output model
class Box(BaseModel):
    """Caputers the size of a box with its units"""
    width: int
    height: int
    depth: int
    units: str

# Define the agent
agent = Agent(model=model,
              result_type=Box,
              system_prompt="Extract me the dimensions of a box",)

# Run the agent
result = agent.run_sync("The box is 10x20x30 cm")

print( f"""
The result is a pydantic data class of type: {type(result.output)}
within the width: {result.output.width} width type: {type(result.output.width)}
within the unit: {result.output.units} units type: {type(result.output.units)}
""")

## Nested Data Structure

## Data extraction from unstructured text

#Output Validators

In [ ]:
import os
from pydantic_ai import Agent, ModelRetry
from pydantic_ai.models.openai import OpenAIModel
from pydantic import BaseModel

# Define the model
model = OpenAIModel('gpt-4o-mini')



# Define the agent
agent = Agent(model=model,
              retries=1,
              system_prompt="see if you have been given all three dimesion to describe the size of a box (width height depth). Answer with a single word either 'correct' or 'wrong'",)

# Define the result validator
@agent.output_validator
def output_validator_simple(data: str) -> str:

    print(f"output_validatorinput data:{data}" )
    if 'wrong' in data.lower():
        raise ModelRetry('wrong response')
    return data

# Run the agent
result = agent.run_sync("The box is 10x20x30 cm")
print(result.output)

result = agent.run_sync("The box is 10")
print(result.output)

## Validate structured Output

## Multiple validators

## Validate executable code